In [5]:
! pip install pretrainedmodels

     |████████████████████████████████| 58 kB 2.9 MB/s eta 0:00:011
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60962 sha256=ff8cef82c75ee143042c8bd1f675631bac61c7b6f1fba23527d1d67a7d43c564
  Stored in directory: /home/ct49/.cache/pip/wheels/ed/27/e8/9543d42de2740d3544db96aefef63bda3f2c1761b3334f4873
Successfully built pretrainedmodels


In [1]:
%matplotlib inline  
# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# data visualisation and manipulation
import numpy as np
import pandas as pd
from matplotlib import style
import seaborn as sns

style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

from sklearn.metrics import confusion_matrix
from fastai import *
from fastai.vision import *
import pandas as pd
import matplotlib.pyplot as plt

# specifically for manipulating zipped images and getting numpy arrays of pixel values of images.
import cv2                  
import numpy as np  
from tqdm import tqdm
import os                   
from random import shuffle  
from zipfile import ZipFile
from PIL import Image
from sklearn.utils import shuffle
from fastai import *
from fastai.vision import *
import torchvision
import pretrainedmodels

In [2]:
from fastai import *
from fastai.vision import *
import pretrainedmodels

In [3]:
train_dir = 'train_set/train_set/'
val_dir = 'val_set/val_set/'

In [4]:
train_df = pd.read_csv('train_labels.csv')
train_df['path'] = train_df['img_name'].map(lambda x: os.path.join(train_dir,x))
val_df = pd.read_csv('val_labels.csv')
val_df['path'] = val_df['img_name'].map(lambda x: os.path.join(val_dir,x))

In [5]:
df = pd.concat([train_df, val_df], ignore_index=True)

In [6]:
val_idx = [i for i in range(len(train_df), len(df))]

In [7]:
sz = 256
bs = 32

In [22]:
import gc
gc.collect()

4873

In [23]:
def top_3_accuracy(preds, targs):
    return top_k_accuracy(preds, targs, 3)

In [120]:
tfms = get_transforms(do_flip=True,flip_vert=True,max_rotate=360,max_warp=0,max_zoom=1.2,max_lighting=0.5,p_lighting=0.5, p_affine=0.5)
src = (ImageList.from_df(df=df,path='./',cols='path') #get dataset from dataset
       .split_by_idx(val_idx)
        .label_from_df(cols='label') #obtain labels from the level column
      )
data= (src.transform(tfms,size=sz,resize_method=ResizeMethod.SQUISH,padding_mode='border') #Data augmentation
        .databunch(bs=bs,num_workers=8) #DataBunch
        .normalize(imagenet_stats) #Normalize
       )

In [113]:
learn = cnn_learner(data, models.densenet169, pretrained=True,metrics=[top_3_accuracy], model_dir='tmp/models', loss_func = LabelSmoothingCrossEntropy()).mixup()

In [115]:
learn.load('stage-2-densenet169')

Learner(data=ImageDataBunch;

Train: LabelList (118475 items)
x: ImageList
Image (3, 256, 256),Image (3, 256, 256),Image (3, 256, 256),Image (3, 256, 256),Image (3, 256, 256)
y: CategoryList
211,211,211,211,211
Path: .;

Valid: LabelList (11994 items)
x: ImageList
Image (3, 256, 256),Image (3, 256, 256),Image (3, 256, 256),Image (3, 256, 256),Image (3, 256, 256)
y: CategoryList
247,234,222,240,218
Path: .;

Test: None, model=Sequential(
  (0): Sequential(
    (0): Sequential(
      (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu0): ReLU(inplace=True)
      (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (denseblock1): _DenseBlock(
        (denselayer1): _DenseLayer(
          (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace=True)
 

In [116]:
learn.fit_one_cycle(5 , slice(1e-5, 1e-3))

epoch,train_loss,valid_loss,top_3_accuracy,time


KeyboardInterrupt: 

In [24]:
def inceptionv4(pretrained=True):
    pretrained = 'imagenet' if pretrained else None
    model = pretrainedmodels.inceptionv4(pretrained=pretrained)
    all_layers = list(model.children())
    return nn.Sequential(*all_layers[0], *all_layers[1:])

In [25]:
def train(arch):
    np.random.seed(42)
    tfms = get_transforms(do_flip=True,flip_vert=True,max_rotate=360,max_warp=0,max_zoom=1.2,max_lighting=0.5,p_lighting=0.5, p_affine=0.5)
    src = (ImageList.from_df(df=df,path='./',cols='path') #get dataset from dataset
       .split_by_idx(val_idx)
        .label_from_df(cols='label') #obtain labels from the level column
      )
    data= (src.transform(tfms,size=sz,resize_method=ResizeMethod.SQUISH,padding_mode='border') #Data augmentation
        .databunch(bs=bs,num_workers=8) #DataBunch
        .normalize(imagenet_stats) #Normalize
       )

    #define learner
    learn = cnn_learner(data, arch, pretrained=True,metrics=[top_3_accuracy], model_dir='tmp/models', loss_func = LabelSmoothingCrossEntropy()).mixup()

    #train
    learn.fit_one_cycle(5, 3e-3)
    learn.unfreeze()
    learn.fit_one_cycle(10 , slice(1e-5, 1e-3))
    
    return learn


In [26]:
# arch1 = = torchvision.models.mobilenet_v2
resnet101_learner = train(models.resnet101)
densenet169_learner = train(models.densenet169)
# vgg_learner = train(models.vgg19_bn)
inception_learner = train(inceptionv4)
# mobileNet_learner = train(arch1)

epoch,train_loss,valid_loss,top_3_accuracy,time
0,4.416145,3.385303,0.528931,17:22
1,4.129720,3.002063,0.638903,17:25
2,4.016320,2.712356,0.711606,17:28
3,3.781898,2.508303,0.754210,17:22
4,3.745818,2.447027,0.767217,17:24


epoch,train_loss,valid_loss,top_3_accuracy,time
0,3.754366,2.495019,0.753960,24:00
1,3.874944,2.631023,0.724279,24:01
2,3.790234,2.610050,0.736452,24:03
3,3.724387,2.444055,0.769051,24:02
4,3.594315,2.483294,0.766133,24:02
5,3.513340,2.228536,0.809571,24:01
6,3.395662,2.158202,0.825496,24:07
7,3.336033,2.086581,0.838753,24:06
8,3.308602,2.053828,0.846340,24:06
9,3.248770,2.045491,0.847924,24:06


epoch,train_loss,valid_loss,top_3_accuracy,time
0,4.396554,3.360419,0.548274,17:58
1,4.148478,3.046239,0.634234,17:58
2,4.003351,2.714945,0.712273,17:55
3,3.761668,2.511030,0.758963,17:54
4,3.711592,2.445077,0.771386,17:55


epoch,train_loss,valid_loss,top_3_accuracy,time
0,3.711575,2.430478,0.774387,23:25
1,3.759429,2.455536,0.768051,23:28
2,3.743883,2.430644,0.771219,23:28
3,3.692979,2.397713,0.780724,23:32
4,3.642935,2.318206,0.797065,23:32
5,3.514469,2.236574,0.815324,23:27
6,3.440114,2.172214,0.825579,23:33
7,3.398919,2.129526,0.831332,23:31
8,3.310593,2.105494,0.839586,23:26
9,3.374574,2.097287,0.840920,23:26


epoch,train_loss,valid_loss,top_3_accuracy,time
0,4.492139,4.177011,0.518259,18:55
1,4.134690,3.019907,0.653160,18:57
2,4.030806,3.026290,0.690929,18:57
3,3.903523,2.712734,0.720193,18:57
4,3.843095,2.612712,0.738453,18:56


epoch,train_loss,valid_loss,top_3_accuracy,time
0,3.778620,2.621173,0.734701,27:07
1,3.775670,2.570636,0.744289,27:11
2,3.758564,2.454986,0.763548,27:13
3,3.669358,2.378296,0.781974,27:13
4,3.579887,2.324331,0.790062,27:14
5,3.529809,2.329760,0.791146,27:13
6,3.541726,2.258862,0.803819,27:14
7,3.492141,2.246728,0.806487,27:12
8,3.492365,2.248691,0.806236,27:10
9,3.465804,2.217425,0.813657,27:13


In [136]:
result_csv = 'submission.csv'
test_path = 'test_set/test_set/'

def ensemble_predition(test_img):
    img = open_image(test_path + test_img)
    
    resnet101_predicition = resnet101_learner.predict(img)
    densenet169_predicition = densenet169_learner.predict(img)
    inception_predicition = inception_learner.predict(img)
    
    #ensemble average
#     sum_pred = resnet101_predicition[2] + resnet101_predicition[2] + resnet101_predicition[2]
    sum_pred = resnet101_predicition[2] + densenet169_predicition[2] + inception_predicition[2]
    prediction = sum_pred / 3
    
    #prediction results
    predicted_label = torch.argsort(prediction,descending = True)[:4]
    predicted_label = str(predicted_label[0].item()) + " " + str(predicted_label[1].item()) +" "+ str(predicted_label[2].item())
#     print(predicted_label)
    return predicted_label

#to give np array the correct style
submission_data = np.array([['dummy', 0]])

#progress bar
with tqdm(total=len(os.listdir(test_path))) as pbar:       
    #test all test images
    for img in os.listdir(test_path):
        label = ensemble_predition(img)
#         print(label)
        new_np_array = np.array([[img, label]])
        submission_data = np.concatenate((submission_data, new_np_array), axis=0)
        pbar.update(1)

#remove dummy
submission_data = np.delete(submission_data, 0, 0)

#save final submission
result_df = pd.DataFrame(submission_data, columns=['img_name','label'])
result_df.to_csv(result_csv, index=False)


100%|██████████| 28377/28377 [53:43<00:00,  8.80it/s] 


In [137]:
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64

def create_download_link(df, title = "Download CSV file", filename = "data.csv"):  
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

In [138]:
create_download_link(result_df, filename='ensemble_models_4_labels.csv')